In [ ]:
!pip install transformers datasets sqlalchemy==2.0 symsearch accelerate bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00


In [ ]:
!gdown --id 1tlkSCzUburJIIJxQLfQFSa07154fZR77
!unzip interim.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1tlkSCzUburJIIJxQLfQFSa07154fZR77
To: /content/interim.zip
100% 733M/733M [00:07<00:00, 97.3MB/s]
Archive:  interim.zip
   creating: interim/test/
  inflating: interim/test/test.json  
   creating: interim/train/
  inflating: interim/train/data-00000-of-00020.json  
  inflating: interim/train/data-00001-of-00020.json  
  inflating: interim/train/data-00002-of-00020.json  
  inflating: interim/train/data-00003-of-00020.json  
  inflating: interim/train/data-00004-of-00020.json  
  inflating: interim/train/data-00005-of-00020.json  
  inflating: interim/train/data-00006-of-00020.json  
  inflating: interim/train/data-00007-of-00020.json  
  inflating: interim/train/data-00008-of-00020.json  
  inflating: interim/train/data-0

# start here

In [ ]:
import torch
import time

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
#Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_datasets = load_dataset('json', data_files='/content/interim/train/*.json', split='train')
train_datasets

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'en_inp', 'en_opt', 'passages', 'retrieve_score', 'rerank_score'],
    num_rows: 267318
})

In [ ]:
en_text = train_datasets[836]
en_text

{'id': '1gakan',
 'en_inp': 'Why the Winter War happened',
 'en_opt': 'Relations between Russia and Finland had been strained since WWI. Russia felt that Finland was weak, and that they would be able to easily seize a decent chunk of territory. Most of the rest of Europe was distracted by Germany gearing up to start WWII, and so the Russians felt that nobody else would really do much to help Finland if they invaded.',
 'passages': ['4614485 ## Russian Winter Russian Winter, General Winter, General Frost, or General Snow refers to the harsh winter climate of Russia as a contributing factor to the military failures of several invasions of Russia. A contributing factor that impairs military maneuvering is "General Mud" ("rasputitsa"), a phenomenon that occurs with autumnal rains and spring thaws in Russia, whereby transport over unimproved roads is made difficult by muddy conditions. Winter as a contributing factor to military defeat In his study of winter warfare in Russia, author Allen 

# vinAI


In [ ]:
vinai_token = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi", src_lang="en_XX")
vinai_model = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi").to(device)

In [ ]:
def translate_vinai(en_texts: str) -> str:
    input_ids = vinai_token(en_texts, padding=True, return_tensors="pt").to(device)
    output_ids = vinai_model.generate(
        **input_ids,
        decoder_start_token_id=vinai_token.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = vinai_token.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [ ]:
# query
translate_vinai(en_text['en_inp'])

['Làm thế nào để máy bay "phá vỡ" rào cản âm thanh?']

In [ ]:
# query
translate_vinai(en_text['en_opt'])

['Tốc độ âm thanh chỉ khoảng 780 dặm một giờ và tất cả những gì bạn cần làm để phá vỡ rào cản âm thanh là đi nhanh hơn thế. Hầu hết các máy bay bình thường đã đi khoảng 500 ~ mph, vì vậy việc tăng tốc lên tới 800 ~ không quá khó']

In [ ]:
# query
translate_vinai(en_text['passages'])

['10675421 ## báo cáo gây ra hầu hết mọi người hình dung rào cản âm thanh như một "bức tường" vật lý, mà máy bay siêu âm cần thiết để "phá vỡ" với một mũi kim nhọn ở mặt trước của thân máy bay. Các sản phẩm của các chuyên gia tên lửa và pháo binh thường xuyên vượt quá Mach 1, nhưng các nhà thiết kế máy bay và kỹ sư khí động học trong và sau Thế chiến II đã thảo luận về Mach 0.7 như một giới hạn nguy hiểm để vượt quá. Những tuyên bố ban đầu Trong Thế chiến II và ngay sau đó, một số tuyên bố đã được đưa ra rằng rào cản âm thanh đã bị phá vỡ trong một lần lặn. Phần lớn các sự kiện có mục đích này có thể bị loại bỏ là lỗi thiết bị đo đạc. Chỉ báo tốc độ không khí điển hình (ASI)',
 "661635 ## máy bay để phá vỡ rào cản âm thanh là một chiếc F-86 Canadair Sabre với phi công phụ nữ'supersonic' đầu tiên, Jacqueline Cochran, tại các điều khiển. Theo David Masters, nguyên mẫu DFS 346 bị Liên Xô bắt ở Đức, sau khi được thả ra từ một chiếc B-29 tại 32800ft (10000 m), đạt 683mph (1100km / h) vào cu

In [ ]:
en_text['passages']

['10675421 ## reports caused most people to envision the sound barrier as a physical "wall", which supersonic aircraft needed to "break" with a sharp needle nose on the front of the fuselage. Rocketry and artillery experts\' products routinely exceeded Mach 1, but aircraft designers and aerodynamic engineers during and after World War II discussed Mach 0.7 as a limit dangerous to exceed. Early claims During WWII and immediately thereafter, a number of claims were made that the sound barrier had been broken in a dive. The majority of these purported events can be dismissed as instrumentation errors. The typical airspeed indicator (ASI)',
 "661635 ## plane to break the sound barrier was an F-86 Canadair Sabre with the first 'supersonic' woman pilot, Jacqueline Cochran, at the controls. According to David Masters, the DFS 346 prototype captured in Germany by the Soviets, after being released from a B-29 at 32800ft (10000 m), reached 683mph (1100km/h) late in 1945, which would have exceede

# VietAI

In [ ]:
vietai_token = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
vietai_model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/envit5-translation").to(device)

In [ ]:
def translate_vietai(en_texts: str) -> str:
    if isinstance(en_texts, list):
      en_texts = ['en: ' + x for x in en_texts]
    else:
      en_texts = 'en: ' + en_texts

    input_ids = vietai_token(en_texts, padding=True, return_tensors="pt").input_ids.to('cuda')
    output_ids = vietai_model.generate(
        input_ids,
        max_length=512
    )
    vi_texts = vietai_token.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [ ]:
translate_vietai(en_text['en_inp'])

['vi: Tại sao chiến tranh mùa đông lại xảy ra']

In [ ]:
translate_vietai(en_text['en_opt'])

['vi: Quan hệ giữa Nga và Phần Lan đã căng thẳng kể từ Thế chiến thứ hai. Nga cảm thấy Phần Lan yếu, và họ có thể dễ dàng chiếm được một phần lãnh thổ kha khá. Phần lớn phần còn lại của châu Âu bị phân tâm bởi Đức đang tăng tốc để bắt đầu Thế chiến thứ hai, và vì vậy Nga cảm thấy không ai khác sẽ thực sự làm nhiều để giúp Phần Lan nếu họ xâm lược.']

In [ ]:
translate_vietai(en_text['passages'])

['vi: 4614485 # # Mùa đông Nga Mùa đông Nga, General Winter, General Frost, hay General Snow đề cập đến khí hậu mùa đông khắc nghiệt của Nga như một yếu tố góp phần vào sự thất bại quân sự của một số cuộc xâm lược Nga.Một yếu tố góp phần làm suy yếu cơ động quân sự là "Mùn chung" ("rasputitsa"), một hiện tượng xảy ra với những cơn mưa mùa thu và tuyết tan vào mùa xuân ở Nga, theo đó việc vận chuyển trên những con đường chưa được cải thiện gặp khó khăn do điều kiện bùn lầy',
 'vi: 4529559 # # trước đợt sương giá nghiêm trọng đầu tiên vào cuối tháng đó và tuyết đầu tiên vào ngày 5 tháng 11. "Lieven trích dẫn khó khăn trong việc tìm kiếm thức ăn cho quân đội và thức ăn cho ngựa vào mùa đông là một yếu tố đóng góp quan trọng. Sự can thiệp của Đồng minh ở Nga, mùa đông 191819 Trong cuộc thám hiểm Bắc Nga của sự can thiệp của Đồng minh trong Nội chiến Nga, cả hai bên, lực lượng Đồng minh và Hồng quân Bolshevik đều biết hoặc nhanh chóng học được các nguyên tắc của chiến tranh mùa đông và áp d

In [ ]:
en_text['passages']

['4614485 ## Russian Winter Russian Winter, General Winter, General Frost, or General Snow refers to the harsh winter climate of Russia as a contributing factor to the military failures of several invasions of Russia. A contributing factor that impairs military maneuvering is "General Mud" ("rasputitsa"), a phenomenon that occurs with autumnal rains and spring thaws in Russia, whereby transport over unimproved roads is made difficult by muddy conditions. Winter as a contributing factor to military defeat In his study of winter warfare in Russia, author Allen F. Chew concludes that "General Winter" was a \'substantial contributing factor\'not a decisive onein the',
 '4529559 ## before the first severe frosts later that month and the first snow on 5 November." Lieven cites the difficulty of finding food for troops and forage for horses in winter as an important contributing factor. Allied intervention in Russia, winter 191819 During the Northern Russian Expedition of the Allied intervent

# NLLB 200 distil

## Setup chung

In [ ]:
source_language={
    "🇱🇷 English": "eng_Latn",
    "🇻🇳 Vietnamese": "vie_Latn",
    "": "empty",
}

## 1.3B


In [ ]:
# weight
weight_path="/media/rick/f7a9be3d-25cd-45d6-b503-7cb8bd32dbd5/pretrained_weights/NLLB/"

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-1.3B", cache_dir=weight_path)

# Load model: CPU: 4.3/12.7GB | GPU: 4.9/15GB
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-1.3B",
    cache_dir=weight_path,
    torch_dtype= torch.float16,
    device_map="auto")
model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [ ]:
def translate_nllb_1B3(text, source_language, target_language):
    """
    Translate the text using the NLLB model.

    Args:
        text (str): Input text.
        source_language (str): Source language code.
        target_language (str): Target language code.

    Returns:
        str: Translated text.
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_language],
        max_length=2048,
    )

    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_text

In [ ]:
translate_nllb_1B3(en_text['en_inp'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['Máy bay làm thế nào để "bẻ gãy" hàng rào âm thanh?']

In [ ]:
translate_nllb_1B3(en_text['en_opt'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['Tốc độ âm thanh chỉ khoảng 780 dặm/giờ, và tất cả những gì bạn cần làm để phá vỡ hàng rào âm thanh là đi nhanh hơn thế.']

In [ ]:
translate_nllb_1B3(en_text['passages'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['Các báo cáo đã khiến hầu hết mọi người hình dung rào cản âm thanh như một "bức tường" vật lý, mà máy bay siêu âm cần phải "bẻ gãy" bằng mũi kim sắc trên mặt trước của thân máy bay. Các sản phẩm của các chuyên gia tên lửa và pháo binh thường xuyên vượt quá Mach 1, nhưng các nhà thiết kế máy bay và kỹ sư khí động học trong và sau Thế chiến II đã thảo luận về Mach 0.7 như một giới hạn nguy hiểm để vượt quá.',
 'Theo David Masters, nguyên mẫu DFS 346 được bắt ở Đức bởi Liên Xô, sau khi được thả khỏi một chiếc B-29 ở 32800ft (10000 m), đạt 683mph (1100km/h) vào cuối năm 1945, mà sẽ vượt quá Mach 1 ở độ cao đó. Phi công trong các chuyến bay này là người Đức Wolfgang Ziese.',
 'Trong khi đó, các máy bay máy bay siêu âm đã được thiết kế theo cách khác nhau, với một số yếu tố khác, các cánh máy bay thông thường đã chứng kiến sự gia tăng đáng kể hệ số kháng lực khi dòng chảy tiếp cận tốc độ âm thanh.',
 '12927672 ## của âm thanh được tiếp cận. Có hai hiệu ứng cụ thể, kéo sóng và máy tính quan 

In [ ]:
en_text['passages']

['10675421 ## reports caused most people to envision the sound barrier as a physical "wall", which supersonic aircraft needed to "break" with a sharp needle nose on the front of the fuselage. Rocketry and artillery experts\' products routinely exceeded Mach 1, but aircraft designers and aerodynamic engineers during and after World War II discussed Mach 0.7 as a limit dangerous to exceed. Early claims During WWII and immediately thereafter, a number of claims were made that the sound barrier had been broken in a dive. The majority of these purported events can be dismissed as instrumentation errors. The typical airspeed indicator (ASI)',
 "661635 ## plane to break the sound barrier was an F-86 Canadair Sabre with the first 'supersonic' woman pilot, Jacqueline Cochran, at the controls. According to David Masters, the DFS 346 prototype captured in Germany by the Soviets, after being released from a B-29 at 32800ft (10000 m), reached 683mph (1100km/h) late in 1945, which would have exceede

# So sánh kết quả


In [ ]:
# weight
weight_path="https://tinyurl.com/nllb200densedst600mcheckpoint"

# Initialize the tokenizer and model
nllb_token_600M = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-1.3B", cache_dir=weight_path)

# Load model: CPU: 4.3/12.7GB | GPU: 4.9/15GB
nllb_model_600M = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    cache_dir=weight_path,
    torch_dtype= torch.float16,
    device_map="auto")
nllb_model_600M.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [ ]:
def translate_nllb_600M(text, source_language, target_language):
    """
    Translate the text using the NLLB model.

    Args:
        text (str): Input text.
        source_language (str): Source language code.
        target_language (str): Target language code.

    Returns:
        str: Translated text.
    """
    inputs = nllb_token_600M(text, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    translated_tokens = nllb_model_600M.generate(
        **inputs,
        forced_bos_token_id=nllb_token_600M.lang_code_to_id[target_language],
        num_return_sequences=1,
        num_beams=5,
    )

    translated_text = nllb_token_600M.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_text

In [ ]:
translate_nllb_600M(en_text['en_inp'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['Máy bay làm thế nào để phá vỡ rào cản âm thanh?']

In [ ]:
translate_nllb_600M(en_text['en_opt'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['Tốc độ âm thanh chỉ là khoảng 780 dặm một giờ, và tất cả những gì bạn cần làm để phá vỡ rào cản âm thanh là đi nhanh hơn thế.']

In [ ]:
translate_nllb_600M(en_text['passages'], source_language['🇱🇷 English'], source_language['🇻🇳 Vietnamese'])

['10675421 # # # báo cáo khiến hầu hết mọi người hình dung rào cản âm thanh như một "vường" vật lý, mà máy bay siêu âm cần phải "lập vỡ" bằng mũi kim sắc nét ở phía trước của thân máy bay. Các sản phẩm của các chuyên gia tên lửa và pháo binh thường vượt quá Mach 1, nhưng các nhà thiết kế máy bay và kỹ sư khí động học trong và sau Thế chiến II thảo luận về Mach 0,7 như là một giới hạn nguy hiểm để vượt qua. Trong Thế chiến II và ngay sau đó, một số tuyên bố đã được đưa ra rằng rào cản âm thanh đã bị phá vỡ trong một cuộc lặn. Phần lớn các sự kiện được cho là này có thể bị loại bỏ là lỗi thiết bị. Chỉ số tốc độ không khí điển hình (ASI)',
 'Chiếc máy bay 661635 ## để phá vỡ rào cản âm thanh là một máy bay F-86 Canadair Sabre với phi công nữ siêu âm đầu tiên, Jacqueline Cochran, ở các điều khiển. Theo David Masters, nguyên mẫu DFS 346 bị Liên Xô bắt giữ ở Đức, sau khi được phát hành từ một chiếc B-29 ở độ cao 32800 ft (10000 m), đạt 683mph (1100km/h) vào cuối năm 1945, mà sẽ vượt quá Mach

In [ ]:
en_text['passages']

['10675421 ## reports caused most people to envision the sound barrier as a physical "wall", which supersonic aircraft needed to "break" with a sharp needle nose on the front of the fuselage. Rocketry and artillery experts\' products routinely exceeded Mach 1, but aircraft designers and aerodynamic engineers during and after World War II discussed Mach 0.7 as a limit dangerous to exceed. Early claims During WWII and immediately thereafter, a number of claims were made that the sound barrier had been broken in a dive. The majority of these purported events can be dismissed as instrumentation errors. The typical airspeed indicator (ASI)',
 "661635 ## plane to break the sound barrier was an F-86 Canadair Sabre with the first 'supersonic' woman pilot, Jacqueline Cochran, at the controls. According to David Masters, the DFS 346 prototype captured in Germany by the Soviets, after being released from a B-29 at 32800ft (10000 m), reached 683mph (1100km/h) late in 1945, which would have exceede